# UrbanSim as input tables 

UrbanSim produces 3 tables:
- Persons 
- Households
- Land use

These table will be manipulated so they are compatible with the the ActivitySim input tables. 

In [1]:
import pandas as pd 
import numpy as np 
import orca 
import time 
from urbansim.utils import misc
import geopandas as gpd
import os; os.chdir('../../..')
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing data  
zones = gpd.read_file('/Users/juandavidcaicedocastro/Desktop/data_UrbanSim/base/Transportation_Analysis_Zones.shp')
zones = zones[['taz1454','district', 'county', 'gacres']]
zones.columns = ['TAZ','district', 'county', 'TOTACRE']
zones.set_index('TAZ', inplace = True)

parcels = pd.read_csv('/Users/juandavidcaicedocastro/Desktop/data_UrbanSim/base/parcels.csv', 
                      index_col='primary_id', dtype={
                    'primary_id': int, 'block_id': str, 'apn': str})
jobs = pd.read_csv('/Users/juandavidcaicedocastro/Desktop/data_UrbanSim/base/jobs.csv',index_col = 'job_id')
buildings = pd.read_csv('/Users/juandavidcaicedocastro/Desktop/data_UrbanSim/base/buildings.csv',index_col = 'building_id')
units = pd.read_csv('/Users/juandavidcaicedocastro/Desktop/data_UrbanSim/base/units.csv',index_col = 'unit_id')
households = pd.read_csv(os.getcwd() +'/bay_area_urbansim/data/raw_households.csv',index_col = 'household_id')
persons = pd.read_csv(os.getcwd() +'/bay_area_urbansim/data/raw_persons.csv',index_col = 'person_id')

parcels.index.rename('parcel_id', inplace = True)

In [3]:
# Land use MTC 
lu_mtc = pd.read_csv(os.getcwd() + '/bay_area_urbansim/data/land_use_mtc.csv', index_col = 'TAZ')

In [4]:
# #Sampling to make it faster 
# buildings = buildings.sample(1000)
# parcels = parcels[parcels.index.isin(buildings.parcel_id)]
# units = units[units.building_id.isin(buildings.index)]
# households = households[households.unit_id.isin(units.index)]
# persons = persons[persons.household_id.isin(households.index)]
# jobs = jobs[jobs.building_id.isin(buildings.index)]

In [5]:
orca.add_table('zones', zones)
orca.add_table('parcels', parcels)
orca.add_table('buildings', buildings)
orca.add_table('units', units)
orca.add_table('households', households)
orca.add_table('persons', persons)
orca.add_table('jobs', jobs)
orca.add_table('lu_mtc', lu_mtc);

# Land use table

Changes in the "annotate_landuse.csv" files as follows:
- TOTHH >> sum_residential)units/(land_use.acres)
- TOTEMP >> total_jobs 
- RESACRE + CIACRE >> acres

Land use table according to https://github.com/BayAreaMetro/modeling-website/wiki/TazData

In [6]:
# Adding TAZ to all tables
@orca.column('buildings')
def TAZ(parcels, buildings):
    return misc.reindex(parcels.zone_id, buildings.parcel_id)

@orca.column('units')
def TAZ(buildings, units):
    return misc.reindex(buildings.TAZ, units.building_id)

@orca.column('households')
def TAZ(units, households):
    return misc.reindex(units.TAZ, households.unit_id)

@orca.column('persons')
def TAZ(households, persons):
    return misc.reindex(households.TAZ, persons.household_id)

#Adding columsn to the zones table
@orca.column('zones')
def COUNTY(zones):
    county_dict = {'San Francisco': 1, 'San Mateo': 2, 'Santa Clara': 3, 
                   'Alameda': 4, 'Contra Costa': 5, 'Solano': 6, 
                   'Napa': 7, 'Sonoma': 8, 'Marin': 9}
    return zones.county.replace(county_dict)

@orca.column('zones', cache=True)
def TOTHH(households, zones):
    s = households.TAZ.groupby(households.TAZ).count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def HHPOP(persons, zones):
    s = persons.TAZ.groupby(persons.TAZ).count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def EMPRES(households, zones):
    s = households.to_frame().groupby('TAZ')['workers'].sum()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def HHINCQ1(households, zones):
    df = households.to_frame()
    df = df[df.income < 30000]
    s = df.groupby('TAZ')['income'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def HHINCQ2(households, zones):
    df = households.to_frame()
    df = df[df.income.between(30000, 59999)]
    s = df.groupby('TAZ')['income'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def HHINCQ3(households, zones):
    df = households.to_frame()
    df = df[df.income .between(60000, 99999)]
    s = df.groupby('TAZ')['income'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def HHINCQ4(households, zones):
    df = households.to_frame()
    df = df[df.income >= 100000]
    s = df.groupby('TAZ')['income'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGE0004(persons, zones):
    df = persons.to_frame()
    df = df[df.age.between(0,4)]
    s = df.groupby('TAZ')['age'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGE0519(persons, zones):
    df = persons.to_frame()
    df = df[df.age.between(5,19)]
    s = df.groupby('TAZ')['age'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGE2044(persons, zones):
    df = persons.to_frame()
    df = df[df.age.between(20,44)]
    s = df.groupby('TAZ')['age'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGE4564(persons, zones):
    df = persons.to_frame()
    df = df[df.age.between(45,64)]
    s = df.groupby('TAZ')['age'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGE65P(persons, zones):
    df = persons.to_frame()
    df = df[df.age >= 65]
    s = df.groupby('TAZ')['age'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGE62P(persons, zones):
    df = persons.to_frame()
    df = df[df.age >= 62]
    s = df.groupby('TAZ')['age'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def SHPOP62P(zones):
    return (zones.AGE62P/zones.HHPOP).reindex(zones.index).fillna(0)

@orca.column('jobs')
def TAZ(buildings, jobs):
    return misc.reindex(buildings.TAZ, jobs.building_id)

@orca.column('zones', cache=True)
def TOTEMP(jobs, zones):
    s = jobs.TAZ.groupby(jobs.TAZ).count()
    return s.reindex(zones.index).fillna(0)

In [7]:
@orca.column('zones', cache=True)
def RETEMPN(jobs, zones):
    df = jobs.to_frame()
    df = df[df.sector_id.isin([44,45])]
    s = df.groupby('TAZ')['sector_id'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def FPSEMPN(jobs, zones):
    df = jobs.to_frame()
    df = df[df.sector_id.isin([52,54])]
    s = df.groupby('TAZ')['sector_id'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def HEREMPN(jobs, zones):
    df = jobs.to_frame()
    df = df[df.sector_id.isin([61, 62, 71])]
    s = df.groupby('TAZ')['sector_id'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def AGREMPN(jobs, zones):
    df = jobs.to_frame()
    df = df[df.sector_id.isin([11])]
    s = df.groupby('TAZ')['sector_id'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def MWTEMPN(jobs, zones):
    df = jobs.to_frame()
    df = df[df.sector_id.isin([42, 31, 32, 33, 48, 49])]
    s = df.groupby('TAZ')['sector_id'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def OTHEMPN(jobs, zones):
    df = jobs.to_frame()
    df = df[~df.sector_id.isin([44,45, 52, 54, 61, 62, 
                              71, 11, 42, 31, 32, 33, 48, 49])]
    s = df.groupby('TAZ')['sector_id'].count()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def RESACRE(parcels, zones):
    df = parcels.to_frame()
    df = df[df.development_type_id.isin([1, 2, 5])]
    s = df.groupby('zone_id')['acres'].sum()
    return s.reindex(zones.index).fillna(0)

@orca.column('zones', cache=True)
def CIACRE(parcels, zones):
    df = parcels.to_frame()
    df = df[df.development_type_id.isin([7, 9, 10, 13, 14,15])]
    s = df.groupby('zone_id')['acres'].sum()
    return s.reindex(zones.index).fillna(0)

This columns are copied from the MTC land use file. 
- TO DO: Figure out a way to calculate this values with our data

In [8]:
# MTC columns
@orca.column('zones')
def PRKCST(lu_mtc):
    return lu_mtc.PRKCST

@orca.column('zones')
def OPRKCST(lu_mtc):
    return lu_mtc.OPRKCST

@orca.column('zones')
def area_type(lu_mtc):
    return lu_mtc.area_type

@orca.column('zones')
def HSENROLL(lu_mtc):
    return lu_mtc.HSENROLL

@orca.column('zones')
def COLLFTE(lu_mtc):
    return lu_mtc.COLLFTE

@orca.column('zones')
def COLLPTE(lu_mtc):
    return lu_mtc.COLLPTE

@orca.column('zones')
def TERMINAL(lu_mtc):
    return lu_mtc.TERMINAL

@orca.column('zones')
def TOPOLOGY(lu_mtc):
    return lu_mtc.TOPOLOGY

In [9]:
zones = orca.get_table('zones').to_frame()

In [10]:
zones.to_csv(os.getcwd() + '/bay_area_urbansim/data/land_use.csv')

To do 

For this columns, figure out a way to calculate its value with our data
- PRKCST 
- OPRKCST
- AREATYPE
- HSENROLL
- COLLFTE
- COLLPTE
- TERMINAL
- TOPOLOGY

In [42]:
# #Development type versus building type
# buildings = pd.read_csv('/Users/juandavidcaicedocastro/Desktop/data_UrbanSim/base/buildings.csv',index_col = 'building_id')
# numbers = list(buildings.development_type_id.sort_values().unique())

# for x in numbers:
#     y = buildings[buildings.development_type_id == x].building_type_id.value_counts()
#     print (x, y)

In [12]:
zones

,district,county,TOTACRE,COUNTY,TOTHH,HHPOP,EMPRES,HHINCQ1,HHINCQ2,HHINCQ3,...,RESACRE,CIACRE,PRKCST,OPRKCST,area_type,HSENROLL,COLLFTE,COLLPTE,TERMINAL,TOPOLOGY
TAZ,,,,,,,,,,,,,,,,,,,,,
566,11,Santa Clara,1708.683056,3,2595.0,6096.0,3262.0,302.0,408.0,561.0,...,259.753381,121.102935,0.0,0.0,4,0.00000,0.0,0.0,1.36118,1
565,11,Santa Clara,486.200669,3,2222.0,5225.0,2694.0,557.0,453.0,465.0,...,161.400641,71.329580,0.0,0.0,4,0.00000,0.0,0.0,1.68135,1
573,11,Santa Clara,538.069933,3,1022.0,3855.0,1872.0,125.0,136.0,263.0,...,155.108047,12.116036,0.0,0.0,4,0.00000,0.0,0.0,1.25867,3
571,11,Santa Clara,450.407906,3,755.0,2989.0,1312.0,114.0,131.0,218.0,...,115.389952,5403.796653,0.0,0.0,4,0.00000,0.0,0.0,1.45307,3
576,11,Santa Clara,423.273023,3,1125.0,4389.0,1957.0,107.0,286.0,258.0,...,170.049777,16.633641,0.0,0.0,4,0.00000,0.0,0.0,1.28300,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,13,Santa Clara,767.051202,3,1740.0,5233.0,2644.0,115.0,194.0,360.0,...,287.896007,27.073028,0.0,0.0,4,0.00000,0.0,0.0,1.19530,3
563,11,Santa Clara,2534.162620,3,2587.0,6872.0,2964.0,981.0,624.0,585.0,...,270.573673,804.566894,0.0,0.0,4,2433.96924,0.0,0.0,1.56422,3
578,11,Santa Clara,424.406441,3,1995.0,7008.0,2884.0,592.0,447.0,533.0,...,172.416750,25.834957,0.0,0.0,3,0.00000,0.0,0.0,1.53180,3


# Households

Changes:

- TAZ : Merging with our zones table (zone > parcel > building > unit) 
- persons changes to PERSONS
- household_id changes to HHID
- cars changes to VEHICL

Probaby check later: HHT variable (HHT = 1 for single_family == False, otherwise HHT = 4; Acording to the constants.py file)

Note: income, workers remain the same 

In [13]:
@orca.step()
def households_table(households):

    @orca.column('households')
    def HHT(households):
        return households.single_family.replace({True: 4, False: 1})
    
    names_dict = {'household_id': 'HHID','persons': 'PERSONS', 'cars': 'VEHICL', 'member_id': 'PNUM'}
    
    df = households.to_frame().rename(columns = names_dict)
    df = df[~df.TAZ.isnull()]
    
    orca.add_table('households', df)

In [14]:
orca.run(['households_table'])

Running step 'households_table'
Time to execute step 'households_table': 4.03 s
Total time to execute iteration 1 with iteration value None: 4.03 s


In [15]:
households = orca.get_table('households').to_frame()

In [16]:
households.to_csv(os.getcwd() +'/bay_area_urbansim/data/households.csv')

# Persons 

In [17]:
@orca.step()
def persons_table(persons):

    @orca.column('persons')
    def ptype(persons):
        #Filters for person type segmentation 
        # https://activitysim.github.io/activitysim/abmexample.html#setup
        age_mask_1 = persons.age >= 18 
        age_mask_2 = persons.age.between(18, 64, inclusive = True)
        age_mask_3 = persons.age >= 65
        work_mask = persons.worker == 1
        student_mask = persons.student == 1

        #Series for each person segmentation 
        type_1 = ((age_mask_1) & (work_mask) & (~student_mask)) * 1 #Full time
        type_4 = ((age_mask_2) & (~work_mask) & (~student_mask)) * 4
        type_5 = ((age_mask_3) & (~work_mask) & (~student_mask)) * 5
        type_3 = ((age_mask_1) & (student_mask)) * 3
        type_6 = (persons.age.between(16, 17, inclusive = True))* 6
        type_7 = (persons.age.between(6, 16, inclusive = True))* 7
        type_8 = (persons.age.between(0, 5, inclusive = True))* 8 
        type_list = [type_1, type_3, type_4, type_5, type_6, type_7, type_8,]

        #Colapsing all series into one series
        for x in type_list:
            type_1.where(type_1 != 0, x, inplace = True)

        return type_1

    @orca.column('persons')
    def pemploy(persons):
        pemploy_1 = ((persons.worker == 1) & (persons.age >= 16)) * 1
        pemploy_3 = ((persons.worker == 0) & (persons.age >= 16)) * 3
        pemploy_4 = (persons.age < 16) * 4

        #Colapsing all series into one series
        type_list = [pemploy_1, pemploy_3, pemploy_4]
        for x in type_list:
            pemploy_1.where(pemploy_1 != 0, x, inplace = True)

        return pemploy_1

    @orca.column('persons')
    def pstudent(persons):
        pstudent_1 = (persons.age <= 18) * 1
        pstudent_2 = ((persons.student == 1) & (persons.age > 18)) * 2
        pstudent_3 = (persons.student == 0) * 3

        #Colapsing all series into one series
        type_list = [pstudent_1, pstudent_2, pstudent_3]
        for x in type_list:
            pstudent_1.where(pstudent_1 != 0, x, inplace = True)

        return pstudent_1
    
    names_dict = {'member_id': 'PNUM'}
    
    df = persons.to_frame().rename(columns = names_dict)
    df = df[~df.TAZ.isnull()]
    
    orca.add_table('persons', df)

In [18]:
orca.run(['persons_table'])
persons = orca.get_table('persons').to_frame()

Running step 'persons_table'
Time to execute step 'persons_table': 9.65 s
Total time to execute iteration 1 with iteration value None: 9.65 s


In [19]:
persons.to_csv(os.getcwd() +'/bay_area_urbansim/data/persons.csv')

In [ ]:
df = pd.read_csv(os.getcwd() +'/bay_area_urbansim/data/persons.csv')
df1 = pd.read_csv(os.getcwd() +'/bay_area_urbansim/data/households.csv')

In [ ]:
# Household id to trace 
# 2015769
# 1841679 solved (Error was that I have university and unemplyment missclassification)

# 926707  solved (density index have some zero value)

In [20]:
persons[(persons.household_id == 926707)]

,PNUM,age,primary_commute_mode,relate,edu,sex,hours,hispanic,earning,race_id,student,work_at_home,worker,household_id,TAZ,ptype,pemploy,pstudent
person_id,,,,,,,,,,,,,,,,,,
2585429,1,65,NaN,0,21.0,2,NaN,1,0.0,2,0,0,0,926707,879,5,3,3
2585430,2,42,NaN,2,21.0,1,40.0,1,398000.0,2,0,0,0,926707,879,4,3,3
2585431,3,13,NaN,7,10.0,1,NaN,1,NaN,2,1,0,0,926707,879,7,4,1
2585432,4,9,NaN,7,6.0,1,NaN,1,NaN,2,1,0,0,926707,879,7,4,1
2585433,5,6,NaN,7,3.0,2,NaN,1,NaN,2,1,0,0,926707,879,7,4,1
2585434,6,55,2.0,10,21.0,2,20.0,1,8000.0,2,0,0,1,926707,879,1,1,3


In [25]:
households.loc[926707]

serialno          2011001456460
PERSONS                       6
building_type                 2
VEHICL                        2
income                   443800
race_of_head                  2
hispanic_head                no
age_of_head                  65
workers                       1
state                         6
county                        1
tract                    410400
block_group                   1
children                      3
tenure                        1
recent_mover                  0
block_group_id      60014104001
single_family              True
unit_id                  390103
building_id              888368
TAZ                         879
HHT                           4
Name: 926707, dtype: object

In [31]:
zones.loc[874, 'CIACRE'] = 2350

In [33]:
zones.to_csv(os.getcwd() + '/bay_area_urbansim/data/land_use.csv')

In [29]:
lu_mtc.loc[874]

DISTRICT       18.00000
SD             18.00000
COUNTY          4.00000
TOTHH           0.00000
HHPOP           0.00000
TOTPOP          0.00000
EMPRES          0.00000
SFDU            0.00000
MFDU            0.00000
HHINCQ1         0.00000
HHINCQ2         0.00000
HHINCQ3         0.00000
HHINCQ4         0.00000
TOTACRE      2428.50000
RESACRE         0.00000
CIACRE       2350.00000
SHPOP62P        0.18300
TOTEMP       3012.00000
AGE0004         0.00000
AGE0519         0.00000
AGE2044         0.00000
AGE4564         0.00000
AGE65P          0.00000
RETEMPN        13.00000
FPSEMPN       677.00000
HEREMPN       456.00000
OTHEMPN       632.00000
AGREMPN         2.00000
MWTEMPN      1232.00000
PRKCST          0.00000
OPRKCST         0.00000
area_type       5.00000
HSENROLL        0.00000
COLLFTE         0.00000
COLLPTE         0.00000
TOPOLOGY        3.00000
TERMINAL        0.95244
ZERO            0.00000
hhlds           0.00000
sftaz         874.00000
gqpop           0.00000
Name: 874, dtype

In [ ]:
df.to_csv(os.getcwd() +'/bay_area_urbansim/data/persons.csv')
df1.to_csv(os.getcwd() +'/bay_area_urbansim/data/households.csv')

# Results

In [ ]:
'/Users/juandavidcaicedocastro/Documents/BERKELEY/08_GSR/activitysim/bay_area_urbansim/output/pipeline.h5'

In [ ]:
os.getcwd()

In [ ]:
#Importing ActivitySim results
hdf = pd.HDFStore(path = os.getcwd() + '/bay_area_urbansim/output/pipeline.h5')
# tours = hdf['/tours/trip_mode_choice']
# trips = hdf['/trips/trip_mode_choice']

In [ ]:
hdf